In [ ]:
import logging
import os
import subprocess
import sys

if os.path.exists("debug.log"):
    os.remove("debug.log")

def custom_format(record):
    match record.levelno:
        case logging.DEBUG:
            level = "🟦"
        case logging.INFO:
            level = "🟩"
        case logging.WARNING:
            level = "🟨"
        case logging.ERROR:
            level = "🟥"
        case logging.CRITICAL:
            level = "🛑"
    return f"{level} {record.getMessage()}"

logger = logging.getLogger()

for handler in logger.handlers:
    logger.removeHandler(handler)

formatter = logging.Formatter()
formatter.format = custom_format

file_handler = logging.FileHandler("debug.log")
file_handler.setFormatter(formatter)
logger.addHandler(file_handler)

stream_handler = logging.StreamHandler()
stream_handler.setFormatter(formatter)
logger.addHandler(stream_handler)

logger.setLevel(logging.DEBUG)

NVIDIA_SMI = subprocess.run(["nvidia-smi"], capture_output=True, text=True).stdout
logging.info(NVIDIA_SMI)
logging.info(f"Python {sys.version}")

In [ ]:
# 1) PyTorchとTransformersのインストール
%pip install torch==2.4.0

# 2) vLLMのインストール
%pip install vllm==0.6.3

# 3) Flash Attentionのインストール
# 2.8.3はundefined symbolエラーが発生するため2.7.3をインストール
# https://github.com/Dao-AILab/flash-attention/issues/1832
# %pip install "https://github.com/Dao-AILab/flash-attention/releases/download/v2.7.3/flash_attn-2.7.3+cu12torch2.6cxx11abiFALSE-cp312-cp312-linux_x86_64.whl" --no-build-isolation
%pip install flash-attn --no-build-isolation

In [ ]:
%cd /workspaces/tinyzero/TinyZero
%pip install -e .

In [ ]:
!python ./examples/data_preprocess/countdown.py --local_dir countdown

In [ ]:
%pip install wandb

In [ ]:
!python3 -m verl.trainer.main_ppo \
data.train_files=$DATA_DIR/train.parquet \
data.val_files=$DATA_DIR/test.parquet \
data.train_batch_size=32 \
data.val_batch_size=32 \
data.max_prompt_length=256 \
data.max_response_length=1024 \
actor_rollout_ref.model.path=$BASE_MODEL \
actor_rollout_ref.model.use_remove_padding=True \
actor_rollout_ref.model.enable_gradient_checkpointing=True \
actor_rollout_ref.actor.use_dynamic_bsz=True \
actor_rollout_ref.actor.optim.lr=1e-6 \
actor_rollout_ref.actor.ppo_mini_batch_size=4 \
actor_rollout_ref.actor.ppo_micro_batch_size=4 \
actor_rollout_ref.rollout.log_prob_micro_batch_size=1 \
actor_rollout_ref.rollout.tensor_model_parallel_size=$ROLLOUT_TP_SIZE \
actor_rollout_ref.rollout.gpu_memory_utilization=0.1 \
actor_rollout_ref.ref.log_prob_micro_batch_size=1 \
critic.optim.lr=1e-5 \
critic.model.path=$BASE_MODEL \
critic.ppo_micro_batch_size=1 \
algorithm.kl_ctrl.kl_coef=0.001 \
trainer.logger=[] \
+trainer.val_before_train=False \
trainer.default_hdfs_dir=null \
trainer.n_gpus_per_node=$N_GPUS \
trainer.nnodes=1 \
trainer.save_freq=100 \
trainer.test_freq=100 \
trainer.project_name=TinyZero \
trainer.experiment_name=$EXPERIMENT_NAME \
trainer.total_epochs=15 2>&1 | tee verl_demo.log


In [ ]:
# export N_GPUS=1
# export BASE_MODEL={path_to_your_model}
# export DATA_DIR={path_to_your_dataset}
# export ROLLOUT_TP_SIZE=1
# export EXPERIMENT_NAME=countdown-qwen2.5-0.5b
# export VLLM_ATTENTION_BACKEND=XFORMERS
# bash ./scripts/train_tiny_zero.sh
!N_GPUS=1 BASE_MODEL="qwen/qwen2.5-0.5b" DATA_DIR="countdown" ROLLOUT_TP_SIZE=1 VLLM_ATTENTION_BACKEND="XFORMERS" bash ./scripts/train_tiny_zero.sh

In [ ]:
# export N_GPUS=2
# export BASE_MODEL={path_to_your_model}
# export DATA_DIR={path_to_your_dataset}
# export ROLLOUT_TP_SIZE=2
# export EXPERIMENT_NAME=countdown-qwen2.5-3b
# export VLLM_ATTENTION_BACKEND=XFORMERS
# bash ./scripts/train_tiny_zero.sh